In [2]:
import pandas as pd
import os
import glob
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [6]:
# Step 1: Load data from all xlsm files in the PENJUALAN folder
folder_path = "./data/Bobby Aquatic 1"  # Ganti dengan path yang sesuai
all_files = glob.glob(os.path.join(folder_path, "*.xlsm"))

In [14]:
dfs = []
for file in all_files:
    df = pd.read_excel(file, sheet_name="Penjualan")
    # Avoid duplicate column names
    if 'KODE BARANG' in df.columns:
        df = df.loc[:, ~df.columns.duplicated()]  # Remove duplicate columns
    dfs.append(df)

# Menggabungkan semua data
data = pd.concat(dfs, ignore_index=True)

In [18]:
# Step 2: Calculate RFM
# Convert 'TANGGAL' to datetime format if necessary
data['TANGGAL'] = pd.to_datetime(data['TANGGAL'])

# Calculate Recency, Frequency, and Monetary
reference_date = data['TANGGAL'].max()  # Set the reference date for Recency
rfm = data.groupby('KODE BARANG').agg({
    'TANGGAL': lambda x: (reference_date - x.max()).days,  # Recency
    'NAMA BARANG': 'count',  # Frequency (use NAMA BARANG or any other column instead)
    'TOTAL HR JUAL': 'sum'  # Monetary (Revenue)
}).reset_index()

# Rename columns
rfm.columns = ['KODE BARANG', 'Recency', 'Frequency', 'Monetary']

# Check the result
print(rfm.head())


  KODE BARANG  Recency  Frequency   Monetary
0         AA1      178          5   560000.0
1        AA10      135          8  2880000.0
2         AA2       83          6  3040000.0
3         AA4      107          5   590000.0
4         AA5       35          6   700000.0


In [20]:
# Step 3: Apply Elbow Method to find optimal number of clusters
# Standardizing the RFM values
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Finding the optimal number of clusters using Elbow Method
sse = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    sse.append(kmeans.inertia_)

KODE BARANG    0
Recency        0
Frequency      0
Monetary       0
dtype: int64
[[ 0.52249566 -0.59600902 -0.60254605]
 [ 0.18808392 -0.50170097  0.00902524]
 [-0.21632096 -0.564573    0.05120257]
 [-0.02967255 -0.59600902 -0.5946378 ]
 [-0.58961778 -0.564573   -0.56564089]]


AttributeError: 'NoneType' object has no attribute 'split'